# Clone Repository and Download Datasets from Dwivedi et al.

In [1]:
!git clone https://github.com/shamim-hussain/benchmarking-gnns.git
%cd benchmarking-gnns/data/
!bash script_download_all_datasets.sh
%cd ..

Cloning into 'benchmarking-gnns'...
remote: Enumerating objects: 882, done.
remote: Total 882 (delta 0), reused 0 (delta 0), pack-reused 882
Receiving objects: 100% (882/882), 2.65 MiB | 1.81 MiB/s, done.
Resolving deltas: 100% (677/677), done.
/home/ml4ig1/Documents code/egt/benchmarking-gnns/data

downloading ZINC.pkl...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    53    0    53    0     0    188      0 --:--:-- --:--:-- --:--:--   188
100   320  100   320    0     0    450      0 --:--:-- --:--:-- --:--:--     0
100 56.1M  100 56.1M    0     0  17.7M      0  0:00:03  0:00:03 --:--:-- 29.7M

downloading ZINC-full.pkl...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    58    0    58    0     0    214      0 --:--:-- --:--:-- --:--:--   214
100   320  10

# Install Libraries

# Convert Datasets

In [1]:
import ogb
%reload_ext autoreload
%autoreload 2

In [9]:
import torch
torch.__version__

'2.0.0+cu117'

In [5]:
from packaging import version
version.parse(torch.__version__) < version.parse("1.13.0")

True

In [ ]:
pytorch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 pytorch-cuda=11.7 pyg=2.4.0 ogb=1.3.6

In [1]:
from ogb.graphproppred import PygGraphPropPredDataset
from ogb.linkproppred import PygLinkPropPredDataset
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
%reload_ext autoreload
%autoreload 2

In [3]:
# Download and process data at './dataset/ogbg_molhiv/'
#dataset = PygLinkPropPredDataset(name='ogbl-ppa',
#                                transform=T.ToSparseTensor())
##split_idx = dataset.get_idx_split() 
#train_loader = DataLoader(dataset[split_idx['train']], batch_size=32, shuffle=True)
#valid_loader = DataLoader(dataset[split_idx['valid']], batch_size=32, shuffle=False)
#test_loader = DataLoader(dataset[split_idx['test']], batch_size=32, shuffle=False)

NameError: name 'split_idx' is not defined

In [5]:
# Download and process data at './dataset/ogbg_molhiv/'
dataset = PygLinkPropPredDataset(name='ogbl-ppa',
                                transform=T.ToSparseTensor())
#split_idx = dataset.get_idx_split() 

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [20]:
data = data.to(device)

In [10]:
data = dataset[0]
data.x = data.x.to(torch.float)
#data.x = torch.cat([data.x, torch.load('embedding.pt')], dim=-1)
data = data.to(device)

split_edge = dataset.get_edge_split()

In [14]:
split_edge = dataset.get_edge_split()

In [22]:
data.adj_t.set_diag()

AttributeError: 'Tensor' object has no attribute 'set_diag'

: 

In [27]:
data.adj_t

tensor(crow_indices=tensor([       0,       11,       22,  ..., 42463821,
                            42463860, 42463862]),
       col_indices=tensor([ 56355, 137379, 194140,  ..., 571433, 135589,
                           173752]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]), device='cuda:0',
       size=(576289, 576289), nnz=42463862, layout=torch.sparse_csr)

In [25]:
adj_t = data.adj_t.to_sparse().set_diag()
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj_t = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
data.adj_t = adj_t

predictor = LinkPredictor(args.hidden_channels, args.hidden_channels, 1,
                        args.num_layers, args.dropout).to(device)

AttributeError: 'Tensor' object has no attribute 'set_diag'

In [13]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.linkproppred import Evaluator

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(GCN, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(
            GCNConv(in_channels, hidden_channels, normalize=False))
        for _ in range(num_layers - 2):
            self.convs.append(
                GCNConv(hidden_channels, hidden_channels, normalize=False))
        self.convs.append(
            GCNConv(hidden_channels, out_channels, normalize=False))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x

class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)


def train(model, predictor, data, split_edge, optimizer, batch_size):
    model.train()
    predictor.train()

    pos_train_edge = split_edge['train']['edge'].to(data.x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):

        optimizer.zero_grad()

        h = model(data.x, data.adj_t)

        edge = pos_train_edge[perm].t()
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        # Just do some trivial random sampling.
        edge = torch.randint(0, data.num_nodes, edge.size(), dtype=torch.long,
                             device=h.device)

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, data, split_edge, evaluator, batch_size):
    model.eval()

    h = model(data.x, data.adj_t)

    pos_train_edge = split_edge['train']['edge'].to(h.device)
    pos_valid_edge = split_edge['valid']['edge'].to(h.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(h.device)
    pos_test_edge = split_edge['test']['edge'].to(h.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(h.device)

    pos_train_preds = []
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size):
        edge = pos_train_edge[perm].t()
        pos_train_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_train_pred = torch.cat(pos_train_preds, dim=0)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    for K in [10, 50, 100]:
        evaluator.K = K
        train_hits = evaluator.eval({
            'y_pred_pos': pos_train_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (train_hits, valid_hits, test_hits)

    return results

In [ ]:
model = GCN(in_channels=data.num_features, hidden_channels=256, out_channels=256, num_layers=3, dropout=0.1).to(device)

In [8]:
for i in train_loader:
    break

In [21]:
i[0]

Data(edge_index=[2, 46], edge_attr=[46, 3], x=[21, 9], y=[1, 1], num_nodes=21)

## SBM_PATTERN

In [3]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self



DATASET_NAME = 'SBM_PATTERN'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

[I] Loading dataset SBM_PATTERN...


ModuleNotFoundError: No module named 'dgl.graph'

In [ ]:
def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp= dgrp.create_group('features/nodes')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['node_labels'] = np.array(l, dtype=int)


dest_file = r'../datasets/SBM_PATTERN/SBM_PATTERN.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('SBM_PATTERN')
    ds.attrs['max_node_feat'] = 2
    ds.attrs['min_node_feat'] = 0
    ds.attrs['num_min_nodes'] = 44
    ds.attrs['num_max_nodes'] = 188
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, trainset)
    save_dataset(val_ds, valset)
    save_dataset(test_ds, testset)

In [ ]:
%reset

## SBM_CLUSTER

In [4]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

DATASET_NAME = 'SBM_CLUSTER'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

[I] Loading dataset SBM_CLUSTER...


ModuleNotFoundError: No module named 'dgl.graph'

In [ ]:
def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp= dgrp.create_group('features/nodes')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['node_labels'] = np.array(l, dtype=int)

dest_file = r'../datasets/SBM_CLUSTER/SBM_CLUSTER.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('SBM_CLUSTER')
    ds.attrs['max_node_feat'] = 6
    ds.attrs['min_node_feat'] = 0
    ds.attrs['num_node_classes'] = 6
    ds.attrs['num_min_nodes'] = 41
    ds.attrs['num_max_nodes'] = 190
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, trainset)
    save_dataset(val_ds, valset)
    save_dataset(test_ds, testset)

In [ ]:
%reset

## MNIST

In [5]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

DATASET_NAME = 'MNIST'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

[I] Loading dataset MNIST...


ModuleNotFoundError: No module named 'dgl.graph'

In [ ]:

def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp,defgrp = dgrp.create_group('features/nodes'), dgrp.create_group('features/edges')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        for fname, fval in g.edata.items():
            defgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['label'] = l.numpy()


dest_file = r'../datasets/MNIST/MNIST.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('MNIST')
    ds.attrs['num_node_feat'] = 3
    ds.attrs['num_edge_feat'] = 1
    ds.attrs['num_classes'] = 10
    ds.attrs['num_min_nodes'] = 40
    ds.attrs['num_max_nodes'] = 75
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, dataset.train)
    save_dataset(val_ds, dataset.val)
    save_dataset(test_ds, dataset.test)

In [ ]:
%reset

## CIFAR10

In [ ]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

DATASET_NAME = 'CIFAR10'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

In [ ]:
def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp,defgrp = dgrp.create_group('features/nodes'), dgrp.create_group('features/edges')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        for fname, fval in g.edata.items():
            defgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['label'] = l.numpy()


dest_file = r'../datasets/CIFAR10/CIFAR10.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('CIFAR10')
    ds.attrs['num_node_feat'] = 5
    ds.attrs['num_edge_feat'] = 1
    ds.attrs['num_classes'] = 10
    ds.attrs['num_min_nodes'] = 85
    ds.attrs['num_max_nodes'] = 150
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, dataset.train)
    save_dataset(val_ds, dataset.val)
    save_dataset(test_ds, dataset.test)

In [ ]:
%reset

## TSP

In [ ]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

DATASET_NAME = 'TSP'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

In [ ]:

def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp,defgrp = dgrp.create_group('features/nodes'), dgrp.create_group('features/edges')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        for fname, fval in g.edata.items():
            defgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['edge_labels'] = np.array(l, dtype=int)

dest_file = r'../datasets/TSP/TSP.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('TSP')
    ds.attrs['num_node_feat'] = 2
    ds.attrs['num_edge_feat'] = 1
    ds.attrs['num_min_nodes'] = 50
    ds.attrs['num_max_nodes'] = 499
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, dataset.train)
    save_dataset(val_ds, dataset.val)
    save_dataset(test_ds, dataset.test)

In [ ]:
%reset

## ZINC

In [ ]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

DATASET_NAME = 'ZINC'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test

In [ ]:


def save_dataset(ds,data):
    for i, (g,l) in tqdm(enumerate(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp,defgrp = dgrp.create_group('features/nodes'), dgrp.create_group('features/edges')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        for fname, fval in g.edata.items():
            defgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['value'] = l.numpy()

dest_file = r'../datasets/ZINC/ZINC.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)

with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('ZINC')
    ds.attrs['num_atom_type'] = 28
    ds.attrs['num_bond_type'] = 4
    ds.attrs['num_min_atoms'] = 9
    ds.attrs['num_max_atoms'] = 37
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, dataset.train)
    save_dataset(val_ds, dataset.val)
    save_dataset(test_ds, dataset.test)

In [ ]:
%reset

## ZINC-full

In [ ]:
import numpy as np
import h5py
import torch
from tqdm import tqdm
from pathlib import Path

from data.data import LoadData

DATASET_NAME = 'ZINC-full'
dataset = LoadData(DATASET_NAME)
trainset, valset, testset = dataset.train, dataset.val, dataset.test #154s

In [ ]:
def save_dataset(ds,data):
    for i, (g,l) in enumerate(tqdm(data)):
        grp = ds.create_group(f'{i:0>10d}')
        dgrp = grp.create_group('data')
        
        dgrp.attrs['num_nodes'] = g.number_of_nodes()
        dgrp.attrs['num_edges'] = g.number_of_edges()
        
        dgrp['edges'] = torch.stack(g.edges(),axis=-1).numpy()
        
        dnfgrp,defgrp = dgrp.create_group('features/nodes'), dgrp.create_group('features/edges')
        for fname, fval in g.ndata.items():
            dnfgrp[fname] = fval.numpy()
        for fname, fval in g.edata.items():
            defgrp[fname] = fval.numpy()
        
        tgrp = grp.create_group('targets')
        tgrp['value'] = l.numpy()

dest_file = r'../datasets/ZINC_full/ZINC_full.h5'
Path(dest_file).parent.mkdir(exist_ok=True, parents=True)
with h5py.File(dest_file, 'w') as file:
    ds = file.create_group('ZINC_full')
    ds.attrs['num_atom_type'] = 28
    ds.attrs['num_bond_type'] = 4
    ds.attrs['num_min_atoms'] = 6
    ds.attrs['num_max_atoms'] = 38
    train_ds, val_ds, test_ds = ds.create_group('training'), ds.create_group('validation'), ds.create_group('test')
    save_dataset(train_ds, dataset.train)
    save_dataset(val_ds, dataset.val)
    save_dataset(test_ds, dataset.test)

In [ ]:
%reset

# Create a Parent HDF with External Soft-links (Optional)

In [ ]:
%cd ../datasets/
%ls

In [ ]:
import h5py as h5

def add_link(data_file, dataset_name):
    data_file[f'{dataset_name}'] = h5.ExternalLink(f"{dataset_name}/{dataset_name}.h5", f"/{dataset_name}")


with h5.File("gnn_benchmark.h5", 'w') as data_file:
    add_link(data_file,"SBM_PATTERN")
    add_link(data_file,"SBM_CLUSTER")
    add_link(data_file,"MNIST")
    add_link(data_file,"CIFAR10")
    add_link(data_file,"TSP")
    add_link(data_file,"ZINC")
    add_link(data_file,"ZINC_full")

## Test the File

In [ ]:
with h5.File("gnn_benchmark.h5", 'r') as data_file:
    datasets = list(data_file.keys())
    print(datasets)
    for dset in datasets:
        print()
        print(dset)
        print('------------')
        print(f'Training examples  : {len(data_file[dset]["training"])}')
        print(f'Validation examples: {len(data_file[dset]["validation"])}')
        print(f'Test examples      : {len(data_file[dset]["test"])}')

# Compress Files for Storage, Download etc. (Optional)

In [ ]:
%cd ../

In [ ]:
!tar -czf datasets.tar.gz datasets

In [ ]:
%ls -alh

## (On Google Colab) Download File

In [ ]:
from google.colab import files

files.download('datasets.tar.gz')

## (On Google Colab) Or Transfer to Google Drive

In [ ]:
from google.colab import drive
drive.mount('./drive')
!cp -v datasets.tar.gz ./drive/MyDrive/
drive.flush_and_unmount()